# ERD - Silver Layer

Original ERD

! [ERD] ("https://lucid.app/lucidchart/8831f2d0-4179-4e75-a5b0-9993800ab551/edit?page=0_0&invitationId=inv_b615b161-c17a-4fcb-b572-66037776abbb#")

Final ERD

! [ERD] ("https://lucid.app/lucidchart/aa4c3cdf-140b-4200-ace7-d76f1aeec0da/edit?page=0_0&invitationId=inv_054df077-327a-40b4-87f6-0a5f18269530")

###imports

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
from pyspark.sql.functions import cast, col, to_timestamp, when, explode,size
from pyspark.sql import DataFrame
from pyspark.sql.functions import broadcast
import os

###functions

# Highlights of our functions
1. Rename payload Columns:
    * Input: given_df -> DataFrame, title -> String
    * Renames the column with the following pattern: parent_child_grandchild
    * Output: return_df -> DataFrame

2. subDF
    * Input: df, title (string)
    * Returns a new DataFrame for the specified column name
    * Output: df

3. subDFWithMasterID / subDFWithoutMasterID / subDFWithMasterID2:
    * Input: df
    * Creates a new df and preserves the specified columns + event_id
    * Output: df

4. add_index:
    * Converts a df into an RDD, creates an index, and outputs the df

In [ ]:
# rename_cols(df.filter(col('payload.issue').isNotNull()), 'issue').display()

def rename_payload_cols(given_df, title):

    # Get the list of all columns under the specified nested column
    nested_columns = given_df.select(f'payload.{title}.*').columns

    # Start with a DataFrame containing the non-nested columns
    new_columns = [f'payload.{title}.{col}' for col in nested_columns]
    return_df = given_df.select('event_id', 'type', *new_columns)

    for column in nested_columns:
        return_df = return_df.withColumnRenamed(column, f'{title}_{column}')

    return return_df


def subDF(given_df, title):
    nested_columns = given_df.select(f'{title}.*').columns
    new_columns = [f'{title}.{col}' for col in nested_columns]
    return_df = given_df.select('event_id', *new_columns)

    for column in nested_columns:
        return_df = return_df.withColumnRenamed(column, f'{title}_{column}')

    return return_df


def subDFWithoutMasterID(given_df, target_table, id_column):
    nested_columns = given_df.select(f'{target_table}.*').columns
    new_columns = [f'{target_table}.{col}' for col in nested_columns]
    return_df = given_df.select('event_id', *new_columns)

def subDFWithMasterID(given_df, target_table, id_column):
    nested_columns = given_df.select(f'{target_table}.*').columns
    new_columns = [f'{target_table}.{col}' for col in nested_columns]
    return_df = given_df.select('event_id', f'master_{id_column}_id', *new_columns)

    for column in nested_columns:
        return_df = return_df.withColumnRenamed(column, f'{target_table}_{column}')

    return return_df


def add_index(df, event_type, start_index = 1):
    df_with_index = df.rdd.zipWithIndex().map(lambda x: (x[1]+start_index, *x[0]))

    schema = StructType([
        StructField(f"{event_type}_id", IntegerType(), False),
        *df.schema.fields
    ])

    df_with_index = df_with_index.toDF(schema=schema)

    return df_with_index


def drop_null_columns(df: DataFrame) -> DataFrame:
    null_columns = [col_name for col_name in df.columns if df.filter(col(col_name).isNotNull()).count() == 0]
    df = df.drop(*null_columns)
    return df


def subDFWithMasterID2(given_df, target_table,*args):
    nested_columns = given_df.select(f'{target_table}.*').columns
    new_columns = [f'{target_table}.{col}' for col in nested_columns]
    return_df = given_df.select('event_id', *new_columns,*args)
    for column in nested_columns:
        return_df = return_df.withColumnRenamed(column, f'{target_table}_{column}')
    return (return_df)


# ADLS Configuration Settings and write to adls function

In [ ]:
adls_container = os.getenv('ADLS_CONTAINER') #azure storage account container
adls_account_name = os.getenv('ADLS_ACCOUNT_NAME')
client_id = os.getenv('ADLS_CLIENT_ID') #aka: app id
tenant_id = os.getenv('ADLS_TENANT_ID') #aka: directory id

service_credential = dbutils.secrets.get(scope="databricks-app-kv",key="databricks-application")

spark.conf.set(f"fs.azure.account.auth.type.{adls_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{adls_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{adls_account_name}.dfs.core.windows.net", f"{client_id}")
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{adls_account_name}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{adls_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

def write_df_to_adls(df, numPartitions, partitionByCol, folder_name , overwrite=False): 
    # Write the DataFrame to ADLS
    df.repartition(numPartitions).write.mode("overwrite" if overwrite else "append").parquet(f'abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/SilverLayer/{folder_name}')


### event_df

In [ ]:
contname = 'team4-project2' #azure storage account container
storage_acct_name = '20230821desa'

file_pattern = f"abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/BronzeLayer/"
df = spark.read.format('parquet').load(f'{file_pattern}').withColumnRenamed('id', 'event_id')

###actor_df

* Collecting all the actor information from main df
* Converting the created at to a timestamp, this will give us the newest user login for each actor id
* Resetting the column names

In [ ]:
# selecting actor columns from df
actor_cols = [col for col in df.columns if col.startswith("actor")]
actor_df = df.select(*actor_cols ,"created_at")

# casting created_at to timestamp
actor_df = actor_df.withColumn('created_at', to_timestamp('created_at'))

# I seleting most updated info per actor_id
new_actor_cols = [col for col in actor_df.columns if col.startswith("actor")]

actor_df = actor_df.groupBy('actor_id').agg({str(new_actor_cols[0]): 'max', str(new_actor_cols[1]): 'max', str(new_actor_cols[2]): 'max', str(new_actor_cols[3]): 'max', str(new_actor_cols[4]): 'max', str(new_actor_cols[5]): 'max', "created_at": "max"}).drop('actor_id', 'max(created_at)')

# renameing max columns
max_actor_cols = [col for col in actor_df.columns if col.startswith("max")]

actor_df = actor_df.withColumnRenamed('max(actor_avatar_url)', 'actor_avatar_url').withColumnRenamed('max(actor_display_login)', 'actor_display_login').withColumnRenamed('max(actor_gravatar_id)', 'actor_gravatar_id').withColumnRenamed('max(actor_url)', 'actor_url').withColumnRenamed('max(actor_id)', 'actor_id').withColumnRenamed('max(actor_login)', 'actor_login')

actor_df = actor_df.select('actor_id', 'actor_login', 'actor_display_login', 'actor_avatar_url', 'actor_url')

actor_df.printSchema()

write_df_to_adls(actor_df, 7, 'actor_id', 'actors' , True)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-173403117478135>:2
      1 # selecting actor columns from df
----> 2 actor_cols = [col for col in df.columns if col.startswith("actor")]
      3 actor_df = df.select(*actor_cols ,"created_at")
      5 # casting created_at to timestamp

NameError: name 'df' is not defined

### org_df

* Collecting all the organization information from main df
* Converting the created at to a timestamp, this will give us the newest organization login for each org id
* Resetting the column names

In [ ]:
# selecting org columns from df
org_cols = [col for col in df.columns if col.startswith("org")]
org_df = df.select("event_id", *org_cols ,"created_at").where(df.org_id.isNotNull())

# casting created_at to timestamp
org_df = org_df.withColumn('created_at', to_timestamp('created_at'))

# I seleting most updated info per actor_id
org_df = org_df.groupBy('org_id').agg({str(org_cols[0]): 'max', str(org_cols[1]): 'max', str(org_cols[2]): 'max', str(org_cols[3]): 'max', str(org_cols[4]): 'max'}).drop('org_id', 'max(created_at)')

# renameing max columns
max_org_cols = [col for col in org_df.columns if col.startswith("max")]

org_df = org_df.withColumnRenamed('max(org_avatar_url)', 'org_avatar_url').withColumnRenamed('max(org_gravatar_id)', 'org_gravatar_id').withColumnRenamed('max(org_id)', 'org_id').withColumnRenamed('max(org_login)', 'org_login').withColumnRenamed('max(org_url)', 'org_url')

org_df = org_df.select('org_id', 'org_login', 'org_url', 'org_avatar_url')

org_df.printSchema()

write_df_to_adls(org_df, 1, 'org_id', 'orgs' , True)

root
 |-- org_id: long (nullable = true)
 |-- org_login: string (nullable = true)
 |-- org_url: string (nullable = true)
 |-- org_avatar_url: string (nullable = true)



###repo_df

* Collecting all the repository information from main df
* Converting the created at to a timestamp, this will give us the newest repo login for each repo id
* Resetting the column names

In [ ]:
# selecting org columns from df
repo_cols = [col for col in df.columns if col.startswith("repo")]
repo_df = df.select("event_id", *repo_cols ,"created_at").where(df.repo_id.isNotNull())

# casting created_at to timestamp
repo_df = repo_df.withColumn('created_at', to_timestamp('created_at'))

# I seleting most updated info per actor_id
repo_df = repo_df.groupBy('repo_id').agg({str(repo_cols[0]): 'max', str(repo_cols[1]): 'max', str(repo_cols[2]): 'max', 'event_id': 'max'}).drop('repo_id', 'max(created_at)')

# renameing max columns
max_repo_cols = [col for col in repo_df.columns if col.startswith("max")]

repo_df = repo_df.withColumnRenamed('max(repo_id)', 'repo_id').withColumnRenamed('max(repo_name)', 'repo_name').withColumnRenamed('max(repo_url)', 'repo_url').withColumnRenamed('max(event_id)', 'event_id')

repo_df = repo_df.select('repo_id', 'repo_name', 'repo_url')

repo_df.printSchema()

write_df_to_adls(repo_df, 9, 'repo_id', 'repo' , True)

root
 |-- repo_id: long (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- repo_url: string (nullable = true)



In [ ]:
df = df.drop('actor_login', 'actor_avatar_url', 'actor_display_login', 'actor_gravatar_id', 'actor_login', 'actor_url' , 'repo_name', 'repo_url', 'org_avatar_url', 'org_gravatar_id', 'org_login', 'org_url')

### master_issues

In [ ]:
master_issues = df.filter(col('payload.issue').isNotNull())
master_issues = rename_payload_cols(master_issues, 'issue')
master_issues = add_index(master_issues, 'master_issue')

### issue_milestone

In [ ]:
# new milestone logic for issues

issue_milestone = master_issues.filter(col('issue_milestone').isNotNull())
issue_milestone = subDFWithMasterID(issue_milestone, 'issue_milestone', 'issue')

issue_milestone = issue_milestone.drop('issue_milestone_creator')

write_df_to_adls(issue_milestone, 1, 'master_issue_id', 'issue_milestone', overwrite= True)


### issue_pull_request

In [ ]:
issue_pull_requests = master_issues.filter(col('issue_pull_request').isNotNull())

issue_pull_requests = subDFWithMasterID(issue_pull_requests, 'issue_pull_request', 'issue')

write_df_to_adls(issue_pull_requests, 4, 'master_issue_id', 'issue_pull_requests', overwrite= True)


### issue_pvga

In [ ]:
issue_pvga = master_issues.filter(col('issue_performed_via_github_app').isNotNull())
issue_pvga = subDFWithMasterID(issue_pvga, 'issue_performed_via_github_app', 'issue')

# create pvga's owner tables and links that with pvga table
issue_pvga = issue_pvga.drop('issue_performed_via_github_app_owner')



# 2 columns in issue PVGA - one is for issue events and the other is for comment events- they can all be diff numbers bc in different columns 

### issue_pvga_permission

In [ ]:
# issue_pvga_permissions = subDF(issue_pvga, 'issue_performed_via_github_app_permissions')

issue_pvga = add_index(issue_pvga, 'master_issue_pvga')

issue_pvga_permissions = issue_pvga.filter(col('issue_performed_via_github_app_permissions').isNotNull())

issue_pvga_permissions = subDFWithMasterID(issue_pvga_permissions, 'issue_performed_via_github_app_permissions', 'issue_pvga')

# # Use these to srater the counts for the next thing ie) pvga in the pull_requests table 

issue_pvga = issue_pvga.drop('issue_performed_via_github_app_permissions')

### issue_pvga_events

In [ ]:
# This is the new issues pvga events setup 
issue_pvga_events = issue_pvga.filter(size(col('issue_performed_via_github_app_events')) > 0)
issue_pvga_events = issue_pvga_events.select('*', explode('issue_performed_via_github_app_events').alias('issue_pvga_events'))

issue_pvga_events = issue_pvga_events.select('issue_pvga_events', 'master_issue_pvga_id')

issue_pvga = issue_pvga.drop('issue_performed_via_github_app_events')


write_df_to_adls(issue_pvga, 1, 'master_issue_id', 'issue_pvga', overwrite= True)
write_df_to_adls(issue_pvga_events, 1, 'master_issue_pvga_id', 'issue_pvga_events', overwrite= True)
write_df_to_adls(issue_pvga_permissions, 1, 'master_issue_pvga_id', 'issue_pvga_permissions', overwrite= True)



### issue_labels

In [ ]:
issue_labels = master_issues.filter(size(col('issue_labels')) > 0).select('*', explode('issue_labels').alias('issue_label'))

issue_labels = subDFWithMasterID(issue_labels, 'issue_label', 'issue')

write_df_to_adls(issue_labels, 4, 'master_issue_id', 'issue_labels', overwrite= True)


### dropping uneeded cols

In [ ]:
master_issues = master_issues.drop('issue_labels', 'issue_pull_request', 'issue_milestone', 'issue_performed_via_github_app', 'issue_reactions', 'issue_assignees', 'issue_user', 'issue_active_lock_reason', 'issue_assignee')




### Comments_master

In [ ]:
comments_master = df.filter(col('payload.comment').isNotNull())
comments_master = rename_payload_cols(comments_master, 'comment')
comments_master = add_index(comments_master, 'master_comment')

### drop comments_user

In [ ]:

comments_master = comments_master.drop('comment_user')



### comments_pvga

In [ ]:
# comments 
comments_pvga = comments_master.filter(col('comment_performed_via_github_app').isNotNull())

comments_pvga = subDFWithMasterID(comments_pvga, 'comment_performed_via_github_app', 'comment')

# create pvga's owner tables and links that with pvga table



comments_pvga = comments_pvga.drop('comment_performed_via_github_app_owner')




### comments_pvga_permissions

In [ ]:
# doing pvga permissions for the comments table

comments_pvga = add_index(comments_pvga, 'master_comments_pvga')

comments_pvga_permissions = comments_pvga.filter(col('comment_performed_via_github_app_permissions').isNotNull())

comments_pvga_permissions = subDFWithMasterID(comments_pvga_permissions, 'comment_performed_via_github_app_permissions', 'comments_pvga')

# merging two permissions tables and setting up the count on permissions

# Use these to srater the counts for the next thing ie) pvga in the pull_requests table 

comments_pvga = comments_pvga.drop('comment_performed_via_github_app_permissions')

write_df_to_adls(comments_pvga_permissions, 1, 'master_comments_pvga_id', 'comments_pvga_permissions', overwrite= True)
write_df_to_adls(comments_pvga, 1, 'master_comment_id', 'comments_pvga', overwrite= True)



### comments_pvga_events

In [ ]:
comments_pvga_events = comments_pvga.filter(size(col('comment_performed_via_github_app_events')) > 0)
comments_pvga_events = comments_pvga_events.select('*', explode('comment_performed_via_github_app_events').alias('comment_pvga_events'))

comments_pvga_events = comments_pvga_events.select('comment_pvga_events', 'master_comments_pvga_id')

comments_pvga = comments_pvga.drop('comment_performed_via_github_app_events')


write_df_to_adls(comments_pvga_events, 1, 'master_comments_pvga_id', 'comments_pvga_events', overwrite= True)





### comments_links 
- pulling out the nested comment__links into master comments table

In [ ]:
comments_master = (
    comments_master.withColumn('comments_link_html', comments_master.comment__links.html.href).withColumn('comments_link_pull_request', comments_master.comment__links.pull_request.href).withColumn('comments_link_self', comments_master.comment__links.self.href).drop('comment__links')
    )

In [ ]:
comments_master = comments_master.drop('comment_performed_via_github_app', 'comment_reactions')



### Pull Requests

In [ ]:
# pull request assignees and table creation with master_id

master_pull_requests = df.filter(col('payload.pull_request').isNotNull())
master_pull_requests = rename_payload_cols(master_pull_requests, 'pull_request')
master_pull_requests = add_index(master_pull_requests, 'master_pull_request')



In [ ]:
master_pull_requests = master_pull_requests.drop('pull_request_user', 'pull_request_merged_by', 'pull_request_requested_reviewers', 'pull_request_assignee', 'pull_request_assignees')


#### Pull Request Milestone

In [ ]:
# new milestone logic for issues



pull_request_milestone = master_pull_requests.filter(col('pull_request_milestone').isNotNull())
pull_request_milestone = subDFWithMasterID(pull_request_milestone, 'pull_request_milestone', 'pull_request')
# pull_request_milestone = pull_request_milestone.withColumn('issue_milestone_id', lit(0))


pull_request_milestone = pull_request_milestone.drop('pull_request_milestone_creator')



write_df_to_adls(pull_request_milestone, 1, 'master_pull_request_id', 'pull_request_milestone', overwrite= True)





#### Pull Request Labels

In [ ]:
# labels logic for pull_request_labels

pull_request_labels = master_pull_requests.filter(size(col('pull_request_labels')) > 0).select('*', explode('pull_request_labels').alias('pull_request_label'))



pull_request_labels = subDFWithMasterID(pull_request_labels, 'pull_request_label', 'pull_request')

write_df_to_adls(pull_request_labels, 4, 'master_pull_request_id', 'pull_request_labels', overwrite= True)




#### Pull Request Links

In [ ]:
# dealing with the links 

master_pull_requests = (
    master_pull_requests.withColumn('pull_request_link_comments', col('pull_request__links.comments.href'))
    .withColumn('pull_request_link_commits', col('pull_request__links.commits.href'))
    .withColumn('pull_request_link_html', col('pull_request__links.html.href'))
    .withColumn('pull_request_link_issue', col('pull_request__links.issue.href'))
    .withColumn('pull_request_link_review_comment', col('pull_request__links.review_comment.href'))
    .withColumn('pull_request_link_review', col('pull_request__links.review_comments.href'))
    .withColumn('pull_request_link_self', col('pull_request__links.self.href'))
    .withColumn('pull_request_link_statuses', col('pull_request__links.statuses.href')).drop('pull_request__links')
)


#### Pull Request Auto Merge

In [ ]:

pull_request_auto_merge = master_pull_requests.filter(col('pull_request_auto_merge').isNotNull())
pull_request_auto_merge = subDFWithMasterID(pull_request_auto_merge, 'pull_request_auto_merge', 'pull_request')

pull_request_auto_merge= pull_request_auto_merge.drop('pull_request_auto_merge_enabled_by')


write_df_to_adls(pull_request_auto_merge, 1, 'master_pull_request_id', 'pull_request_auto_merge', overwrite= True)





#### Pull Request Base

In [ ]:
pull_request_base = master_pull_requests.filter(col('pull_request_base').isNotNull())
pull_request_base = subDFWithMasterID(pull_request_base, 'pull_request_base', 'pull_request')

pull_request_base = add_index(pull_request_base, 'master_pull_request_base')





#### Pull Request Repo

In [ ]:
# create milestone's creator tables and links that with milestone table
pull_request_base_repo = pull_request_base.filter(col('pull_request_base_repo').isNotNull())
pull_request_base_repo = subDFWithMasterID(pull_request_base_repo, 'pull_request_base_repo', 'pull_request_base')
pull_request_base_repo = add_index(pull_request_base_repo, 'master_pull_request_repo_base')

pull_request_base_repo= pull_request_base_repo.drop('pull_request_base_repo_owner')




#### Pull Request Topics

In [ ]:
pull_request_base_repo_topics = pull_request_base_repo.filter(size(col('pull_request_base_repo_topics')) > 0)
pull_request_base_repo_topics = pull_request_base_repo_topics.select('*', explode('pull_request_base_repo_topics').alias('pull_request_base_repo_topic'))

pull_request_base_repo_topics = pull_request_base_repo_topics.select('master_pull_request_base_id', 'pull_request_base_repo_topic')

pull_request_base_repo = pull_request_base_repo.drop('pull_request_base_repo_topics')
pull_request_base = pull_request_base_repo.drop('pull_request_base_repo')



#### Pull Request Base Links

In [ ]:

pull_request_base_repo = (
    pull_request_base_repo.withColumn('pull_request_base_repo_license_key', col('pull_request_base_repo_license.key')).withColumn('pull_request_base_repo_license_name', col('pull_request_base_repo_license.name')).withColumn('pull_request_base_repo_license_node_id', col('pull_request_base_repo_license.node_id')).withColumn('pull_request_base_repo_license_spdx_id', col('pull_request_base_repo_license.spdx_id')).withColumn('pull_request_base_repo_license_url', col('pull_request_base_repo_license.url')).drop('pull_request_base_repo_license')
)






#### Writitng Pull Request Base Tables

In [ ]:

write_df_to_adls(pull_request_base, 122, 'master_pull_request_id', 'pull_request_base', overwrite= True)
write_df_to_adls(pull_request_base_repo, 122, 'master_pull_request_base_id', 'pull_request_base_repo', overwrite= True)
write_df_to_adls(pull_request_base_repo_topics, 1, 'master_pull_request_repo_base_id', 'pull_request_base_repo_topics', overwrite= True)


#### Pull Request Head

In [ ]:
pull_request_head = master_pull_requests.filter(col('pull_request_head').isNotNull())
pull_request_head = subDFWithMasterID(pull_request_head, 'pull_request_head', 'pull_request')
pull_request_head = add_index(pull_request_head, 'master_pull_request_head')


#### Pull Request Head Repo

In [ ]:
# create milestone's creator tables and links that with milestone table
pull_request_head_repo = pull_request_head.filter(col('pull_request_head_repo').isNotNull())
pull_request_head_repo = subDFWithMasterID(pull_request_head_repo, 'pull_request_head_repo', 'pull_request')
pull_request_head_repo = add_index(pull_request_head_repo, 'master_pull_request_head_repo')

pull_request_head_repo = pull_request_head_repo.drop('pull_request_head_repo_owner')




#### Pull Request Head Topics

In [ ]:
pull_request_head_repo_topics = pull_request_head_repo.filter(size(col('pull_request_head_repo_topics')) > 0)
pull_request_head_repo_topics = pull_request_head_repo_topics.select('*', explode('pull_request_head_repo_topics').alias('pull_request_head_repo_topic'))

pull_request_head_repo_topics = pull_request_head_repo_topics.select('master_pull_request_head_repo_id', 'pull_request_head_repo_topic')

pull_request_head_repo = pull_request_head_repo.drop('pull_request_head_repo_topics')
pull_request_head = pull_request_head_repo.drop('pull_request_head_repo')



#### Pull Request Head Repo License

In [ ]:

pull_request_head_repo = (
    pull_request_head_repo.withColumn('pull_request_head_repo_license_key', col('pull_request_head_repo_license.key')).withColumn('pull_request_head_repo_license_name', col('pull_request_head_repo_license.name')).withColumn('pull_request_head_repo_license_node_id', col('pull_request_head_repo_license.node_id')).withColumn('pull_request_head_repo_license_spdx_id', col('pull_request_head_repo_license.spdx_id')).withColumn('pull_request_head_repo_license_url', col('pull_request_head_repo_license.url')).drop('pull_request_head_repo_license')
)



#### Writing Pull Request Head Tables

In [ ]:

write_df_to_adls(pull_request_head, 122, 'master_pull_request_id', 'pull_request_head', overwrite= True)
write_df_to_adls(pull_request_head_repo, 122, 'master_pull_request_head_id', 'pull_request_head_repo', overwrite= True)
write_df_to_adls(pull_request_head_repo_topics, 1, 'master_pull_request_repo_head_id', 'pull_request_head_repo_topics', overwrite= True)


#### Pull Request Requested Teams

In [ ]:
pull_request_requested_teams = master_pull_requests.filter(size(col('pull_request_requested_teams')) > 0)
pull_request_requested_teams = pull_request_requested_teams.select('*', explode('pull_request_requested_teams').alias('pull_request_requested_team'))

pull_request_requested_teams = subDFWithMasterID(pull_request_requested_teams, 'pull_request_requested_team', 'pull_request')

pull_request_requested_teams = add_index(pull_request_requested_teams, 'master_pull_request_requested_teams')

pull_request_requested_teams_parent = pull_request_requested_teams.filter(col('pull_request_requested_team_parent').isNotNull())
# pull_request_requested_teams_parent = subDFWithMasterID(pull_request_requested_teams, 'pull_request_requested_team_parent', 'pull_request')

pull_request_requested_teams_parent= subDFWithMasterID(pull_request_requested_teams, 'pull_request_requested_team_parent', 'pull_request_requested_teams')

pull_request_requested_teams = pull_request_requested_teams.drop('pull_request_requested_team_parent')

write_df_to_adls(pull_request_requested_teams, 1, 'master_pull_request_id', 'pull_request_requested_teams', overwrite= True)
write_df_to_adls(pull_request_requested_teams_parent, 1, 'master_pull_request_requested_teams_id', 'pull_request_requested_teams_parent', overwrite= True)



#### Dropping Unecessary Cols in Pull Requests

In [ ]:

master_pull_requests = master_pull_requests.drop('pull_request_requested_teams', 'pull_request_head', 'pull_request_base', 'pull_request_auto_merge', 'pull_request_requested_reviewers', 'pull_request_labels', 'pull_request_milestone', 'pull_request_assignees', 'pull_request_reactions', 'pull_request__links')



### Master_reviews

In [ ]:
master_reviews = df.filter(col('payload.review').isNotNull())
master_reviews = rename_payload_cols(master_reviews, 'review')
master_reviews = add_index(master_reviews, 'master_reviews')


### Renaming license table and dropping cols

In [ ]:

master_reviews = master_reviews.withColumn('review_link_html', col('review__links.html.href')).withColumn('review_link_pull_request', col('review__links.pull_request.href')).drop('review__links')

master_reviews = master_reviews.drop('review_reactions', 'review_user')



### Writing issues, comments, review and pull_request tables 

In [ ]:
# todo 


write_df_to_adls(master_issues, 86, 'master_issues_id', 'master_issues', overwrite= True)
write_df_to_adls(comments_master, 37, 'comment_master_id', 'comments_master', overwrite= True)
write_df_to_adls(master_pull_requests, 166, 'master_pull_requests_id', 'master_pull_requests', overwrite= True)
write_df_to_adls(master_reviews, 5, 'master_reviews_id', 'master_reviews', overwrite= True)





In [ ]:

# Original Logic

# issue_users = master_issues.filter(col('issue_user').isNotNull())
# issue_users = subDF(issue_users, 'issue_user')

# issue_users = add_index(issue_users, 'user', master_users_count).withColumnRenamed('event_id', 'issue_event_id')


# # Linking the issue_users with the master user's table and addding new values to the count
# master_users = master_users.union(issue_users).drop_duplicates(subset = ['issue_assignee_id'])


# master_users_count = master_users.count()

# # linking user_issues with master_issues table and setting null values to 0 
# master_issues = (
#             master_issues.join(broadcast(master_users), master_issues.issue_user.id 
#             == master_users.issue_assignee_id, how= 'left')
#  )

# master_issues.cache()

# master_issues = master_issues.withColumnRenamed('user_id', 'users_id').drop(*master_users.columns)
# master_issues = master_issues.fillna(0, subset = ['users_id'])

# master_issues.unpersist()


### Fork Event

In [ ]:
fork_event_df = df.where(df.type == 'ForkEvent')
fork_event_df = rename_payload_cols(fork_event_df, 'forkee')

fork_event_df = df.select('event_id', 'payload.forkee').where(df.type == 'ForkEvent')

# unnesting reviews object
fork_event_df = subDF(fork_event_df, 'forkee')

# setting FK for other tables
fork_event_df = fork_event_df.withColumn('forkee_license', fork_event_df.forkee_license.node_id).withColumn('forkee_owner', fork_event_df.forkee_owner.id).withColumn('forkee_topics_count', size(col('forkee_topics'))).drop('forkee_topics')

# fork_event_df.display()

### Fork License

In [ ]:
# selecting fork event rows
fork_license_df = df.select('event_id', 'payload.forkee.license').where(df.type == 'ForkEvent')
fork_license_df = subDF(fork_license_df, 'license').drop('event_id').dropDuplicates(['license_node_id']).dropna(how='all')
fork_license_df = add_index(fork_license_df, 'license')

fork_license_mapper = fork_license_df.select('license_node_id')
fork_license_mapper = add_index(fork_license_mapper, 'forkee_license')

fork_event_df = fork_event_df.join(fork_license_mapper, fork_event_df.forkee_license == fork_license_mapper.license_node_id, 'left')

#write_df_to_adls(fork_license_df, 1, 'license_id', 'fork_license', True)
fork_events_df = fork_event_df.drop('license_node_id', 'forkee_license')

# fork_license_df.printSchema()

In [ ]:
# fork_events_df.printSchema()

### Fork Owner

In [ ]:
# selecting fork event rows
fork_owner_df = df.select('event_id', 'payload.forkee.owner').where(df.type == 'ForkEvent')
fork_owner_df = subDF(fork_owner_df, 'owner').drop('event_id')

# fork_owner_df.printSchema()

In [ ]:
write_df_to_adls(fork_event_df, 23, 'event_id', 'fork_event', True)
write_df_to_adls(fork_owner_df, 23, 'owner_id', 'fork_owner', True)
write_df_to_adls(fork_license_df, 1, 'forkee_license_id', 'fork_license', True)


### Watch Event

In [ ]:
watch_events = df.select('event_id', 'payload.action').where(df.type == 'WatchEvent').withColumn('event_id', df.event_id.cast(LongType()))

# watch_events.printSchema()

write_df_to_adls(watch_events, 4, 'event_id', 'watch_event', True)


### Member Events

In [ ]:
member_event_df = df.select('event_id', 'payload.action', 'payload.member.id').where(df.type == 'MemberEvent').withColumnRenamed('id', 'member_id')

write_df_to_adls(member_event_df, 1, 'event_id', 'member_event', True)

# member_event_df.printSchema()

### Member Users


In [ ]:
# selecting member event rows
member_users = df.select('event_id', 'payload.member').where(df.type == 'MemberEvent')
member_users = subDF(member_users, 'member').drop('event_id')

# member_users.printSchema()

write_df_to_adls(member_event_df, 10, 'event_id', 'member_user', True) # need to determine partitions num

# Azim's 

### Create Event

* Created a new table called Create Event.
* Used event_id from main table from events data since there is a 1 to 1 relationship

In [ ]:
df_ce = df.filter(df.type=="CreateEvent")
df_ce = subDFWithMasterID2(df_ce,"payload")
df_ce = drop_null_columns(df_ce)

In [ ]:
df_ce.printSchema()

In [ ]:
write_df_to_adls(df_ce,10,"event_id", 'create_event',True)


In [ ]:
#dbutils.notebook.exit("Create Event finished")
print("Create Event finished")

### Delete Event

* Created a new df called delete event
* Used event_id from parent table to preserve 1 to 1 relationship

In [ ]:
df_del = df.filter(df.type=="DeleteEvent")
df_del = drop_null_columns(subDFWithMasterID2(df_del,"payload"))

In [ ]:
df_del.printSchema()

In [ ]:
write_df_to_adls(df_del,10,"event_id",'delete_event',True)

In [ ]:
#TODO
# Save the df_del to the parquet file. This has details about all the DeleteEvents
#dbutils.notebook.exit("Delete Event finished")
print("Delete Event finished")

### Gollum Event

* Broken down the pages information and assigned unique id to each page
* Preserved event_id to have the 1 to many relationship

In [ ]:
df_gol = df.filter(df.type=="GollumEvent")
df_gol = subDFWithMasterID2(df_gol,"payload")
df_gol = drop_null_columns(df_gol)
df_pages_gol = df_gol.select("event_id",explode("payload_pages").alias("payload_pages"))
df_pages_gol = add_index(df_pages_gol,"gollum_page")
df_pages_gol = subDFWithMasterID2(df_pages_gol,"payload_pages","gollum_page_id")

In [ ]:
df_gol.printSchema()

In [ ]:
df_pages_gol.printSchema()

In [ ]:
write_df_to_adls(df_pages_gol,1,"gollum_page_id","gollum_page",True)

In [ ]:
#TODO
#Save the df df_pages_gol to a parquet file
#dbutils.notebook.exit("Gollum Event finished")
print("Gollum Event finished")

### Release Event

* Preserves the event_id to have the one on one relationship with main events table
* breaks down the author payload data into its different df. Subs author id instead of author information for normalization
* Breaks down assets into its own df and leaves the asset id information to preserve the 1 to many relationship

In [ ]:
df_release = df.filter(df.type=="ReleaseEvent")
df_release = drop_null_columns(subDFWithMasterID2(df_release,"payload"))
df_release = subDFWithMasterID2(df_release,"payload_release","payload_action")
payload_release_assets = df_release.select("event_id",explode("payload_release_assets").alias("payload_release_assets"))
payload_release_assets = subDFWithMasterID2(payload_release_assets,"payload_release_assets")
df_release = df_release.withColumn("payload_release_assets_id",col("payload_release_assets.id"))
df_release = df_release.drop("payload_release_assets")
payload_release_author = subDFWithMasterID2(df_release,"payload_release_author")
df_release = df_release.withColumn("payload_release_author_id",col("payload_release_author.id"))
df_release = df_release.drop("payload_release_author")
col = "event_id"
if col in payload_release_assets.columns:
    payload_release_assets = payload_release_assets.drop(col)

if col in payload_release_author.columns:
    payload_release_author = payload_release_author.drop(col)

In [ ]:
#TODO
#df_release write parquet
write_df_to_adls(df_release,10,"event_id", 'release_event',True)
#payload_release_assets write parquet
write_df_to_adls(payload_release_assets,10,"event_id", 'payload_release_assets',True)
#payload_release_assets_uploader write parque
#payload_release_author write parquet
write_df_to_adls(payload_release_author,10,"event_id", 'release_author',True)


In [ ]:
df_release.printSchema()

root
 |-- event_id: string (nullable = true)
 |-- payload_release_assets_url: string (nullable = true)
 |-- payload_release_body: string (nullable = true)
 |-- payload_release_created_at: string (nullable = true)
 |-- payload_release_discussion_url: string (nullable = true)
 |-- payload_release_draft: boolean (nullable = true)
 |-- payload_release_html_url: string (nullable = true)
 |-- payload_release_id: long (nullable = true)
 |-- payload_release_is_short_description_html_truncated: boolean (nullable = true)
 |-- payload_release_mentions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- avatar_url: string (nullable = true)
 |    |    |-- avatar_user_actor: boolean (nullable = true)
 |    |    |-- login: string (nullable = true)
 |    |    |-- profile_name: string (nullable = true)
 |    |    |-- profile_url: string (nullable = true)
 |-- payload_release_mentions_count: long (nullable = true)
 |-- payload_release_name: string (nullable = true)
 |

In [ ]:
payload_release_assets.printSchema()

root
 |-- event_id: string (nullable = true)
 |-- payload_release_assets_browser_download_url: string (nullable = true)
 |-- payload_release_assets_content_type: string (nullable = true)
 |-- payload_release_assets_created_at: string (nullable = true)
 |-- payload_release_assets_download_count: long (nullable = true)
 |-- payload_release_assets_id: long (nullable = true)
 |-- payload_release_assets_label: string (nullable = true)
 |-- payload_release_assets_name: string (nullable = true)
 |-- payload_release_assets_node_id: string (nullable = true)
 |-- payload_release_assets_size: long (nullable = true)
 |-- payload_release_assets_state: string (nullable = true)
 |-- payload_release_assets_updated_at: string (nullable = true)
 |-- payload_release_assets_uploader: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- events_url: string (nullable = true)
 |    |-- followers_url: string (nullable = true)
 |    |-- following_url: string (nullable = true)
 |    

In [ ]:
payload_release_author.printSchema()

root
 |-- event_id: string (nullable = true)
 |-- payload_release_author_avatar_url: string (nullable = true)
 |-- payload_release_author_events_url: string (nullable = true)
 |-- payload_release_author_followers_url: string (nullable = true)
 |-- payload_release_author_following_url: string (nullable = true)
 |-- payload_release_author_gists_url: string (nullable = true)
 |-- payload_release_author_gravatar_id: string (nullable = true)
 |-- payload_release_author_html_url: string (nullable = true)
 |-- payload_release_author_id: long (nullable = true)
 |-- payload_release_author_login: string (nullable = true)
 |-- payload_release_author_node_id: string (nullable = true)
 |-- payload_release_author_organizations_url: string (nullable = true)
 |-- payload_release_author_received_events_url: string (nullable = true)
 |-- payload_release_author_repos_url: string (nullable = true)
 |-- payload_release_author_site_admin: boolean (nullable = true)
 |-- payload_release_author_starred_url: st


### Push Event 

* Breaks down the push event by filter
* breaks the commit down in its own table
* Preserves the one to many relationship in commit vs main table

In [ ]:
df_push = df.filter(df.type=="PushEvent")
df_push = subDFWithMasterID2(df_push,"payload")
df_push = drop_null_columns(df_push)
df_commits = df_push.select("event_id",explode("payload_commits").alias("commits"))
df_commits = add_index(df_commits,"push_commit")
df_commits = subDFWithMasterID2(df_commits,"commits","push_commit_id")
df_commits = subDFWithMasterID2(df_commits,"commits_author","push_commit_id","commits_distinct","commits_message","commits_sha","commits_url")
df_push = df_push.drop("commits","distinct_size")
df_commits.drop("event_id")

DataFrame[commits_author_email: string, commits_author_name: string, push_commit_id: int, commits_distinct: boolean, commits_message: string, commits_sha: string, commits_url: string]

In [ ]:
df_push.printSchema

In [ ]:
write_df_to_adls(df_push,13,"event_id","push_event",True)
write_df_to_adls(df_commits,7,"push_commit_id","push_commits",True)

# Commit Comment Event

* filters out the payload Comments
* Breaks down the comment users in its own tables 
* Preserves the relationship by the user id

In [ ]:
df_cce = df.filter(df.type=="CommitCommentEvent")
df_cce = drop_null_columns(subDFWithMasterID2(df_cce,"payload"))
df_cce = subDFWithMasterID2(df_cce,"payload_comment")
df_cce = drop_null_columns(df_cce)
df_cce_users = subDFWithMasterID2(df_cce,"payload_comment_user")
df_cce = df_cce.withColumn("payload_comment_user_id",col("payload_comment_user.id"))
df_cce = df_cce.drop("payload_comment_user")

In [ ]:
#TODO
#df_cce_users parquet
write_df_to_adls(df_cce,10,"event_id", 'commit_comment_event',True)
write_df_to_adls(df_cce,10,"event_id", 'commit_comment_event_users',True)

#df_cce parquet

# Public Event

In [ ]:
df = df.withColumn("made_public_via_event",when(col("type") == "Public Event", True).otherwise(False))


Saving Main Table

In [ ]:


df = df.drop('payload')

write_df_to_adls(df,23,"event_id",'master_events',True)

In [ ]:
df.printSchema()